In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string


In [2]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['class'] = 0
true['class'] = 1
data = pd.concat([fake,true], axis =0)

In [6]:
data = data.drop(["title","subject","date"], axis = 1)
data.sample(10)

,text,class
22641,21st Century Wire says Did Mitt Romney just ma...,0
13531,"Facebook CEO and founder, Mark Zuckerberg is c...",0
8496,When you think about the sacrifice our men and...,0
16501,"CLARK, Philippines (Reuters) - The timing of U...",1
6446,WASHINGTON (Reuters) - Republican U.S. Senator...,1
13943,BEIJING (Reuters) - China has leverage to pers...,1
17169,SEOUL (Reuters) - South Korea is considering l...,1
20110,Crooked lying Hillary,0
12387,,0
5754,LOS ANGELES/NEW YORK (Reuters) - A federal jud...,1


In [7]:
data.reset_index(inplace=True)
data.drop(["index"],axis= 1, inplace = True)


In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r"https?://\S+www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\w*\d\w*", "", text)
    return text



In [9]:
data["text"] = data["text"].apply(clean_text)

In [10]:
x = data["text"]
y = data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)

In [11]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)



In [12]:
# Training a Logistic Regression model
lr = LogisticRegression()
lr.fit(xv_train, ytrain)

# Making predictions
prediction = lr.predict(xv_test)

# Evaluating the model


In [13]:
print(lr.score(xv_test, ytest))  # Prints accuracy
print(classification_report(ytest, prediction))  # Prints detailed classification report

0.9860133630289533
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [14]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr.jb")

['lr.jb']